In [7]:
import pandas as pd
import numpy as np
from keras.layers import Activation, Dense, Dropout, SpatialDropout1D,Input,Masking,Bidirectional, TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM,GRU
from keras.models import Sequential, Model
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from random import seed
from sklearn.metrics import roc_auc_score
#seed(1)
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model
import scipy.stats
from prettytable import PrettyTable
import math
import itertools
from random import shuffle


#defining the early stopping criteria
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,restore_best_weights=True,patience=3)

gs = pd.read_csv('data_8th_june.csv')
gs.drop('Unnamed: 0',axis=1,inplace=True)
gs.drop('Unnamed: 0.1',axis=1,inplace=True)



def split_70(x):
    return int((round((x/15)*0.7))*15)


def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return round(m,3), round(m-h,3), round(m+h,3)



def range_finder(x):
    length = x
    fractional = (x/15.0) - math.floor(x/15.0)
    return int(round(fractional*15))

def make_lstm(gd):


    final_df = pd.DataFrame(columns=gd.columns)
    for i in gd.uhid.unique():
        x = gd[gd['uhid']==i]
        x = x[range_finder(len(x)):len(x)]

        final_df = final_df.append(x,ignore_index=True)



    final_df.fillna(-999,inplace=True)


    # In[ ]:


    train = final_df[:split_70(len(final_df))]
    test = final_df[split_70(len(final_df)):]



    # In[ ]:


    y_train = train['dischargestatus']
    X_train = train.drop('dischargestatus',axis=1)
    X_train = X_train.drop('uhid',axis=1)
    #X_train = X_train.drop('visittime',axis=1)

    y_test = test['dischargestatus']
    X_test = test.drop('dischargestatus',axis=1)
    X_test = X_test.drop('uhid',axis=1)
    #X_test = X_test.drop('startdate',axis=1)


    # In[ ]:

    auc_roc_inter = []
    val_a = []
    train_a = []


    #converting the data into a numpy array
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    ytrain1 = []
    for i in range(0,len(y_train),15):
        #print(i)
        y1 = y_train[i:i+15]
        ytrain1.append(y1[-1])

    ytest1 = []
    for i in range(0,len(y_test),15):
        #print(i)
        y1 = y_test[i:i+15]
        ytest1.append(y1[-1])

    ytrain1 = np.array(ytrain1)
    ytest1 = np.array(ytest1)

    Xtrain = np.reshape(X_train, (-1, 15, X_train.shape[1]))
    Xtest = np.reshape(X_test, (-1, 15, X_test.shape[1]))

    return Xtrain,Xtest,ytrain1,ytest1

def lstm_model(n,Xtrain,Xtest,ytrain1,ytest1):
    auc_roc_inter = []
    val_a = []
    train_a = []
    for i in range(2):
        #Building the LSTM model
        X = Input(shape=(None, n), name='X')
        mX = Masking()(X)
        lstm = Bidirectional(LSTM(units=512,activation='tanh',return_sequences=True,recurrent_dropout=0.5,dropout=0.3))
        mX = lstm(mX)
        L = LSTM(units=64,activation='tanh',return_sequences=False)(mX)
        y = Dense(1, activation="sigmoid")(L)
        outputs = [y]
        inputs = [X]
        model = Model(inputs,outputs)
        model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])

        v_a = []
        t_a = []
        #fitting the model
        model.fit(Xtrain, ytrain1, batch_size=60 ,validation_split=0.15,epochs=38,callbacks=[es])
        #history = model.fit(Xtrain, ytrain1, batch_size=60 ,validation_split=0.15,epochs=38,callbacks=[es])
        for i in range(len(model.history.history['val_accuracy'])):
            v_a.append(model.history.history['val_accuracy'][i])
            t_a.append(model.history.history['accuracy'][i])
        #predictions
        y_pred = model.predict(Xtest)
        #y_pred = y_pred.round()
        y_test = np.array(ytest1)
        y_pred = np.array(y_pred)
        y_test = pd.DataFrame(y_test)
        y_test = np.array(y_test)

        def acc(x):
            if x>0.5:
                return 1
            else:
                return 0

        y_model=[]
        for i in y_pred:
            y_model.append(acc(i))
        y_answer=[]
        for j in y_test:
            y_answer.append(acc(j))
            
        val_a.append(v_a)
        train_a.append(t_a)
        auc_roc_inter.append(roc_auc_score(y_answer,y_pred))
        continue
    
    
        
    return auc_roc_inter,y_model,y_answer




In [8]:
cols_to_use = ['uhid','heartrate', 'spo2', 'ecg_resprate' ,'dischargestatus']
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model((len(cols_to_use)-2),Xtrain,Xtest,ytrain1,ytest1)
a_a = mean_confidence_interval(an)

Train on 29203 samples, validate on 5154 samples
Epoch 1/38
29203/29203 [==============================] - 50s 2ms/step - loss: 0.2286 - accuracy: 0.9145 - val_loss: 0.2790 - val_accuracy: 0.9018
Epoch 2/38
29203/29203 [==============================] - 49s 2ms/step - loss: 0.2157 - accuracy: 0.9196 - val_loss: 0.2235 - val_accuracy: 0.9012
Epoch 3/38
29203/29203 [==============================] - 49s 2ms/step - loss: 0.2170 - accuracy: 0.9194 - val_loss: 0.2645 - val_accuracy: 0.9020
Epoch 4/38
29203/29203 [==============================] - 49s 2ms/step - loss: 0.2127 - accuracy: 0.9222 - val_loss: 0.2546 - val_accuracy: 0.8999
Epoch 5/38
29203/29203 [==============================] - 48s 2ms/step - loss: 0.2158 - accuracy: 0.9191 - val_loss: 0.2492 - val_accuracy: 0.9014
Restoring model weights from the end of the best epoch
Epoch 00005: early stopping
Train on 29203 samples, validate on 5154 samples
Epoch 1/38
29203/29203 [==============================] - 50s 2ms/step - loss: 0.226

In [13]:
cols_to_use = ['uhid','heartrate','spo2', 'dischargestatus']
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model((len(cols_to_use)-2),Xtrain,Xtest,ytrain1,ytest1)
a_b = mean_confidence_interval(an)

Train on 29203 samples, validate on 5154 samples
Epoch 1/38
29203/29203 [==============================] - 50s 2ms/step - loss: 0.6471 - accuracy: 0.5988 - val_loss: 0.6435 - val_accuracy: 0.7823
Epoch 2/38
29203/29203 [==============================] - 48s 2ms/step - loss: 0.6441 - accuracy: 0.6068 - val_loss: 0.6820 - val_accuracy: 0.5254
Epoch 3/38
29203/29203 [==============================] - 48s 2ms/step - loss: 0.6402 - accuracy: 0.6102 - val_loss: 0.6987 - val_accuracy: 0.5109
Epoch 4/38
29203/29203 [==============================] - 48s 2ms/step - loss: 0.6417 - accuracy: 0.6109 - val_loss: 0.7697 - val_accuracy: 0.3343
Restoring model weights from the end of the best epoch
Epoch 00004: early stopping
Train on 29203 samples, validate on 5154 samples
Epoch 1/38
29203/29203 [==============================] - 50s 2ms/step - loss: 0.6486 - accuracy: 0.6006 - val_loss: 0.6725 - val_accuracy: 0.5561
Epoch 2/38
29203/29203 [==============================] - 48s 2ms/step - loss: 0.642

In [14]:
cols_to_use = ['uhid','spo2','ecg_resprate', 'dischargestatus']
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model((len(cols_to_use)-2),Xtrain,Xtest,ytrain1,ytest1)
a_c = mean_confidence_interval(an)

Train on 29203 samples, validate on 5154 samples
Epoch 1/38
29203/29203 [==============================] - 50s 2ms/step - loss: 0.2960 - accuracy: 0.8832 - val_loss: 0.2086 - val_accuracy: 0.9020
Epoch 2/38
29203/29203 [==============================] - 48s 2ms/step - loss: 0.2892 - accuracy: 0.8854 - val_loss: 0.2159 - val_accuracy: 0.9020
Epoch 3/38
29203/29203 [==============================] - 48s 2ms/step - loss: 0.2864 - accuracy: 0.8874 - val_loss: 0.2145 - val_accuracy: 0.9020
Epoch 4/38
29203/29203 [==============================] - 48s 2ms/step - loss: 0.2854 - accuracy: 0.8867 - val_loss: 0.2169 - val_accuracy: 0.9020
Restoring model weights from the end of the best epoch
Epoch 00004: early stopping
Train on 29203 samples, validate on 5154 samples
Epoch 1/38
29203/29203 [==============================] - 50s 2ms/step - loss: 0.2963 - accuracy: 0.8824 - val_loss: 0.2181 - val_accuracy: 0.9018
Epoch 2/38
29203/29203 [==============================] - 48s 2ms/step - loss: 0.288

In [15]:
cols_to_use = ['uhid','heartrate','ecg_resprate', 'dischargestatus']
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model((len(cols_to_use)-2),Xtrain,Xtest,ytrain1,ytest1)
a_d = mean_confidence_interval(an)

Train on 29203 samples, validate on 5154 samples
Epoch 1/38
29203/29203 [==============================] - 50s 2ms/step - loss: 0.2239 - accuracy: 0.9168 - val_loss: 0.2797 - val_accuracy: 0.9009
Epoch 2/38
29203/29203 [==============================] - 54s 2ms/step - loss: 0.2162 - accuracy: 0.9200 - val_loss: 0.2822 - val_accuracy: 0.9005
Epoch 3/38
29203/29203 [==============================] - 73s 2ms/step - loss: 0.2161 - accuracy: 0.9213 - val_loss: 0.2618 - val_accuracy: 0.9007
Epoch 4/38
29203/29203 [==============================] - 61s 2ms/step - loss: 0.2142 - accuracy: 0.9203 - val_loss: 0.2598 - val_accuracy: 0.9001
Epoch 5/38
29203/29203 [==============================] - 51s 2ms/step - loss: 0.2191 - accuracy: 0.9188 - val_loss: 0.3184 - val_accuracy: 0.8999
Epoch 6/38
29203/29203 [==============================] - 71s 2ms/step - loss: 0.2154 - accuracy: 0.9184 - val_loss: 0.2680 - val_accuracy: 0.9016
Epoch 7/38
29203/29203 [==============================] - 62s 2ms/ste

In [16]:
l = [["HR, RR, SpO2" ,a_a],["HR SpO2" ,a_b],["RR SpO2" ,a_c],["HR RR" ,a_d]]

table = PrettyTable(['Parameter', 'AUC-ROC'])

for rec in l:
    table.add_row(rec)

print(table)

+--------------+-----------------------+
|  Parameter   |        AUC-ROC        |
+--------------+-----------------------+
| HR, RR, SpO2 | (0.939, 0.891, 0.986) |
|   HR SpO2    |  (0.65, 0.647, 0.653) |
|   RR SpO2    | (0.852, 0.296, 1.408) |
|    HR RR     | (0.932, 0.923, 0.941) |
+--------------+-----------------------+
